In [1]:
import pandas as pd
import re

In [2]:
warn_nasdaq_df = pd.read_csv(
    "../data/warn_jaccard_0.5.csv",
    delimiter=","
)
warn_nasdaq_df = warn_nasdaq_df.rename({"id2": "Symbol"}, axis=1)
warn_nasdaq_df = warn_nasdaq_df.drop_duplicates(subset=["OG-Company-Warn", "Symbol"])
warn_nasdaq_df = warn_nasdaq_df.sort_values("jaccard", ascending=True)
warn_nasdaq_df.to_csv("../warn_jaccard_0.5_dedup.csv", index=False)

In [3]:
warn_nasdaq_cleaned_df = pd.read_csv("../data/warn_jaccard_0.5_cleaned.csv")
warn_nasdaq_cleaned_df = warn_nasdaq_cleaned_df.drop(["id1", "joinKey1", "joinKey2", "jaccard"], axis=1)
warn_nasdaq_cleaned_df = warn_nasdaq_cleaned_df.drop_duplicates(subset=["OG-Company-Warn", "Symbol"])
warn_nasdaq_cleaned_df = warn_nasdaq_cleaned_df.set_index("OG-Company-Warn")
print(warn_nasdaq_cleaned_df['Symbol'].unique().shape)
warn_nasdaq_cleaned_df.head(10)

(579,)


,Symbol,OG-Company-Nasdaq
OG-Company-Warn,,
Credit Suisse Asset Management LLC (CSAM),CIK,Credit Suisse Asset Management Income Fund Inc...
"G-III Apparel Group, Ltd. (3 NYC sites)",GIII,G-III Apparel Group LTD. Common Stock
Royal Caribbean Cruises Ltd.,RCL,D/B/A Royal Caribbean Cruises Ltd. Common Stock
M&T Bank (Mid-Hudson),MTB,M&T Bank Corporation Common Stock
"Delta Air Lines, Inc-Pilot Base",DAL,Delta Air Lines Inc. Common Stock
JP Morgan Chase & Co.-San Antonio,JPM,JP Morgan Chase & Co. Common Stock
Bristol-Myers Squibb and MyoKardia Inc.,BMY,Bristol-Myers Squibb Company Common Stock
"Keurig Dr Pepper Inc.; Keurig Green Mountain, Inc.",KDP,Keurig Dr Pepper Inc. Common Stock
"Live Nation Entertainment, Inc Corporate Division",LYV,Live Nation Entertainment Inc. Common Stock


In [4]:
# warn_layoffs_df = pd.read_csv(
#     "../data_preparation/scraping/warn.csv",
#     index_col="id"
# )
warn_layoffs_df = pd.read_csv(
    "../data/warn_1.csv",
    # dtype='string',
    parse_dates=["WARN Received Date", "Effective Date"]
)
# warn_layoffs_df = warn_layoffs_df.drop_duplicates(subset=["Company", "State", "WARN Received Date", "Number of Workers"])
warn_layoffs_df = warn_layoffs_df[[
    "State", "Company", "City", "Number of Workers", "WARN Received Date", "Effective Date"
]]
warn_layoffs_df = warn_layoffs_df.drop_duplicates()
print(warn_layoffs_df.shape)
warn_layoffs_df.head(10)

(44136, 6)


/var/folders/z2/drm596w97t97nxjngc05sxk00000gn/T/ipykernel_63099/3887228810.py:5: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  warn_layoffs_df = pd.read_csv(


,State,Company,City,Number of Workers,WARN Received Date,Effective Date
0,Alabama,"Primus Solutions, LLC",Daleville,132,2021-07-28,9/16/2021
1,Alabama,Corsicana Mattress Company,Jasper,42,2021-07-26,9/20/2021
2,Alabama,Portfolio Recovery Associates LLC,Birmingham,133,2021-07-20,9/30/2021
3,Alabama,Crothall Healthcare Environmental Services,Gadsden,58,2021-07-07,8/21/2021
4,Alabama,National Packaging Co. Inc.,Decatur,62,2021-06-30,8/7/2021
5,Alabama,Eberspaecher,Northport,47,2021-06-21,8/17/2021
6,Alabama,Imagen Brands,Mobile,70,2021-01-26,4/1/2021
7,Alabama,Technicolor,Huntsville,275,2021-01-26,6/1/2021
8,Alabama,Dow Chemical Company,Mount Meigs,55,2020-12-03,12/31/2020
9,Alabama,DRiV Automotive,Jacksonville,62,2020-11-18,1/22/2021


In [5]:
merged_df = warn_layoffs_df.join(other=warn_nasdaq_cleaned_df, how="inner", on="Company")
merged_df = merged_df.reset_index(drop=True)
merged_df["Date"] = merged_df["WARN Received Date"].fillna(merged_df["Effective Date"])
merged_df = merged_df.astype({'Date': 'datetime64[ns]'})
merged_df = merged_df.drop(["WARN Received Date", "Effective Date", "Company"], axis=1)
merged_df = merged_df.rename({"OG-Company-Nasdaq": "Company"}, axis=1)
merged_df = merged_df[~merged_df["Number of Workers"].isna()]
merged_df = merged_df.sort_values(by="Date", ascending=True)

merged_df = merged_df.groupby(
    by=["State", "City", "Symbol", "Company", "Date"],
    as_index=False
).sum()

print(merged_df.shape)
merged_df.head(10)
# len(merged_df['Symbol'].unique().tolist())
# merged_df[merged_df["Number of Workers"].isna()]
# merged_df[merged_df['Symbol']=='BA']
# warn_layoffs_df[warn_layoffs_df['Company']=='THE BOEING COMPANY']

(2874, 6)


,State,City,Symbol,Company,Date,Number of Workers
0,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2009-04-10,90
1,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2012-06-11,134
2,Alabama,Athens,OC,Owens Corning Inc Common Stock New,2002-01-30,63
3,Alabama,Athens,PPC,Pilgrim's Pride Corporation Common Stock,2009-07-24,640
4,Alabama,Bessemer,CAH,Cardinal Health Inc. Common Stock,2008-04-24,100
5,Alabama,Birmingham,CAH,Cardinal Health Inc. Common Stock,2006-08-08,49
6,Alabama,Birmingham,FRBA,First Bank Common Stock,2008-04-09,78
7,Alabama,Birmingham,KBR,KBR Inc. Common Stock,2020-10-19,111
8,Alabama,Birmingham,T,AT&T Inc.,2013-03-15,67
9,Alabama,Birmingham,T,AT&T Inc.,2017-04-18,134


In [6]:
fmp_df = pd.read_csv(
    "../data/combined_company_financials.csv",
)
fmp_df = fmp_df.rename({'date': 'statement_date'}, axis=1).astype({'statement_date': 'datetime64[ns]'})
fmp_df = fmp_df.set_index("symbol", drop=True).sort_index(na_position="first")
print(fmp_df.shape)
fmp_df.head(10)
# print(fmp_df.columns.to_list())

(61389, 96)


,statement_date,revenue,costOfRevenue,grossProfit,grossProfitRatio,ResearchAndDevelopmentExpenses,GeneralAndAdministrativeExpenses,SellingAndMarketingExpenses,otherExpenses,operatingExpenses,...,dividendsPaid,otherFinancingActivites,netCashUsedProvidedByFinancingActivities,effectOfForexChangesOnCash,netChangeInCash,cashAtEndOfPeriod,cashAtBeginningOfPeriod,operatingCashFlow,capitalExpenditure,freeCashFlow
symbol,,,,,,,,,,,,,,,,,,,,,
A,2001-10-31,1.660000e+09,1.337000e+09,323000000.0,0.194578,339000000.0,642000000.0,0.0,0.0,981000000.0,...,0.0,-788000000.0,-770000000.0,0.0,1.710000e+08,1.170000e+09,9.990000e+08,1.273000e+09,-167000000.0,1.106000e+09
A,2008-04-30,1.456000e+09,6.490000e+08,807000000.0,0.554258,183000000.0,433000000.0,0.0,0.0,616000000.0,...,0.0,250000000.0,2000000.0,14000000.0,3.250000e+08,1.710000e+09,1.385000e+09,3.250000e+08,-37000000.0,2.880000e+08
A,2008-07-31,1.444000e+09,6.410000e+08,803000000.0,0.556094,170000000.0,415000000.0,0.0,0.0,585000000.0,...,0.0,-40000000.0,-176000000.0,-3000000.0,-7.000000e+07,1.640000e+09,1.710000e+09,1.690000e+08,-39000000.0,1.300000e+08
A,2008-10-31,1.481000e+09,6.510000e+08,830000000.0,0.560432,170000000.0,408000000.0,0.0,0.0,578000000.0,...,0.0,-194000000.0,-432000000.0,-33000000.0,-2.350000e+08,1.405000e+09,1.640000e+09,2.580000e+08,-44000000.0,2.140000e+08
A,2009-01-31,1.166000e+09,5.770000e+08,589000000.0,0.505146,169000000.0,396000000.0,0.0,0.0,565000000.0,...,0.0,50000000.0,-49000000.0,0.0,-4.300000e+07,1.362000e+09,1.405000e+09,1.700000e+07,-34000000.0,-1.700000e+07
A,2009-04-30,1.091000e+09,5.610000e+08,530000000.0,0.485793,170000000.0,407000000.0,0.0,0.0,577000000.0,...,0.0,-50000000.0,-81000000.0,0.0,3.800000e+07,1.400000e+09,1.362000e+09,1.370000e+08,-34000000.0,1.030000e+08
A,2009-07-31,1.057000e+09,5.180000e+08,539000000.0,0.509934,153000000.0,387000000.0,0.0,0.0,540000000.0,...,0.0,0.0,26000000.0,18000000.0,7.900000e+07,1.479000e+09,1.400000e+09,4.100000e+07,-30000000.0,1.100000e+07
A,2009-10-31,1.167000e+09,5.330000e+08,634000000.0,0.543273,150000000.0,413000000.0,0.0,0.0,563000000.0,...,0.0,743000000.0,761000000.0,6000000.0,1.000000e+09,2.479000e+09,1.479000e+09,2.130000e+08,-30000000.0,1.830000e+08
A,2010-01-31,1.213000e+09,5.530000e+08,660000000.0,0.544106,149000000.0,417000000.0,0.0,0.0,566000000.0,...,0.0,0.0,3000000.0,0.0,2.000000e+06,2.481000e+09,2.479000e+09,3.000000e+07,-25000000.0,5.000000e+06


In [7]:
warn_fmp_df = merged_df.join(other=fmp_df, on="Symbol", how="inner")
warn_fmp_df = warn_fmp_df[warn_fmp_df['statement_date'] < warn_fmp_df['Date']]
warn_fmp_df = warn_fmp_df[(warn_fmp_df['Date'] - warn_fmp_df['statement_date']) <= pd.Timedelta("365 days")]
# warn_fmp_df[] = warn_fmp_df['statement_date'] - warn_fmp_df['Date']
warn_fmp_df = warn_fmp_df[[
    'State', 'City', 'Symbol', 'Company', 'Date', 'Number of Workers', 'statement_date', 'revenue', 'netIncome', 'EPS'
]]
warn_fmp_df = warn_fmp_df.reset_index(drop=True)
warn_fmp_df = warn_fmp_df.drop_duplicates(
    subset=['State', 'City', 'Symbol', 'Company', 'Date', 'Number of Workers', 'statement_date'])
print(warn_fmp_df.shape)
warn_fmp_df.head()

(11094, 10)


,State,City,Symbol,Company,Date,Number of Workers,statement_date,revenue,netIncome,EPS
0,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2012-06-11,134,2011-07-31,2.596000e+09,178000000.0,0.541033
1,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2012-06-11,134,2011-10-31,2.811000e+09,-91000000.0,-0.270000
2,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2012-06-11,134,2012-01-31,2.492000e+09,-161000000.0,-0.489362
3,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2012-06-11,134,2012-04-30,2.782000e+09,117000000.0,0.355623
4,Alabama,Huntsville,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2010-12-10,202,2010-10-31,2.821000e+09,171000000.0,0.470000


In [8]:
new_df = None
grouped = warn_fmp_df.groupby(by=['State', 'City', 'Symbol', 'Company', 'Date', 'Number of Workers'])
for name, group in grouped:
    if group['statement_date'].shape[0] == 4:
        group['statement_date'] = ['t-1','t-2','t-3','t-4']
        if new_df is None:
            new_df = group
        else:
            new_df = pd.concat([new_df, group], ignore_index=True)
    # print(group.shape)
# warn_fmp_df
warn_fmp_df = new_df

In [9]:
# pivoted_df = warn_fmp_df[['State', 'City', 'Symbol', 'Company', 'Date', 'statement_date', 'revenue', 'costOfRevenue']]
pivoted_df = warn_fmp_df.drop(["Number of Workers"], axis=1)
pivoted_df = pivoted_df.set_index(['State', 'City', 'Symbol', 'Company', 'Date', 'statement_date'])
# pivoted_df.apply()
# pivoted_df = warn_fmp_df.set_index(['State', 'City', 'Symbol', 'Company', 'Date', 'Number of Workers'])
pivoted_df = pivoted_df.unstack().reset_index()
pivoted_df.columns = pivoted_df.columns.map(lambda c: ":".join(filter(len, c)))
# pivoted_df.columns = pivoted_df.columns.to_flat_index()
pivoted_df

,State,City,Symbol,Company,Date,revenue:t-1,revenue:t-2,revenue:t-3,revenue:t-4,netIncome:t-1,netIncome:t-2,netIncome:t-3,netIncome:t-4,EPS:t-1,EPS:t-2,EPS:t-3,EPS:t-4
0,Alabama,Anniston,SAIC,SCIENCE APPLICATIONS INTERNATIONAL CORPORATION...,2012-06-11,2.596000e+09,2.811000e+09,2.492000e+09,2.782000e+09,178000000.0,-91000000.0,-1.610000e+08,117000000.0,0.541033,-0.270000,-0.489362,0.355623
1,Alabama,Athens,OC,Owens Corning Inc Common Stock New,2002-01-30,1.165000e+09,1.291000e+09,1.239000e+09,1.067000e+09,-7000000.0,27000000.0,2.900000e+07,-10000000.0,-0.127042,0.500000,0.530000,-0.190000
2,Alabama,Athens,PPC,Pilgrim's Pride Corporation Common Stock,2009-07-24,2.169489e+09,1.876991e+09,1.698102e+09,1.776813e+09,-802926000.0,-228782000.0,-5.876500e+07,53239000.0,-10.842190,-3.090000,-0.790000,0.720000
3,Alabama,Bessemer,CAH,Cardinal Health Inc. Common Stock,2008-04-24,2.290960e+10,2.226290e+10,2.328270e+10,2.197340e+10,356000000.0,902200000.0,3.247000e+08,301800000.0,1.000000,2.485399,0.910000,0.830000
4,Alabama,Birmingham,CAH,Cardinal Health Inc. Common Stock,2006-08-08,2.063750e+10,2.170830e+10,1.991900e+10,1.937340e+10,146800000.0,321000000.0,3.040000e+08,228300000.0,0.350000,0.793572,0.710000,0.540000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2723,Wisconsin,Racine,XRAY,DENTSPLY SIRONA Inc. Common Stock,2019-03-27,1.059700e+09,9.284000e+08,1.042100e+09,9.561000e+08,1800000.0,28000000.0,-1.122000e+09,81200000.0,0.010000,0.130000,-4.980000,0.360000
2724,Wisconsin,Ripon,SJM,J.M. Smucker Company (The) New Common Stock,2022-03-08,1.920200e+09,2.057100e+09,2.050000e+09,1.858000e+09,147000000.0,69700000.0,2.060000e+08,153900000.0,1.350000,0.640000,1.900000,1.420000
2725,Wisconsin,Tomah,SND,Smart Sand Inc. Common Stock,2020-04-28,4.766700e+07,6.569000e+07,6.794100e+07,4.748800e+07,2388000.0,10926000.0,1.427600e+07,-84000.0,0.060000,0.270000,0.360000,-0.002095
2726,Wisconsin,Wausau,DLX,Deluxe Corporation Common Stock,2021-04-01,4.104050e+08,4.394610e+08,4.544930e+08,4.412640e+08,14859000.0,29417000.0,2.466300e+07,24292000.0,0.360000,0.701788,0.588474,0.580000
